In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv(r"F:\baidudownload\ml-20m\ml-20m\movies.csv")

In [4]:
ratings = pd.read_csv(r'F:\baidudownload\ml-20m\ml-20m\ratings.csv')

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [10]:
data = pd.merge(movies,ratings,on='movieId',how='left')

In [12]:
data[['userId','rating','movieId','title']].sort_values('userId').to_csv(r"F:\baidudownload\ml-20m\ml-20m\data.csv",index=False)

### 采用python字典来表示每位用户评论的电影和评分

In [25]:
files = open(r"F:\baidudownload\ml-20m\ml-20m\data.csv",'r',encoding="UTF-8")
# 读取data文件中每行中除了名字的数据
data = {} ## 存放每个用户评论的电影和评分
for line in files.readlines():
    line = line.strip().split(',')
    # 如果字典中没有某位用户，则使用用户ID来创建这位用户
    if not line[0] in data.keys():
        data[line[0]] = {line[3]:line[1]} # 子字典
    else:
         data[line[0]][line[3]] = line[1]

## 计算任何两位用户之间的相似度，由于每位用户评论的电影不完全一样，所以兽先要找到两位用户共同评论过的电影然后计算两者之间的欧式距离，最后算出两者之间的相似度

In [16]:
from math import *

In [38]:
def Euclidean(user1,user2):
    # 取出两位用户评论过的电影和评分
    user1_data = data[user1]
    user2_data = data[user2]
    
    # 找到两位用户都评论过的电影，并计算两者的欧式距离
    for key in user1_data.keys():
        if key in user2_data.keys():
#             print(user1_data[key],user2_data[key])
            try:
                distance +=pow((float(user1_data[key])-float(user2_data[key])),2)
            except:
                print("error:",user2_data[key])
    return 1/(1+sqrt(distance)) # 计算返回值越小，相似度越大

In [39]:
# 计算某个用户与其他用的相似度
def top10_simliar(userID):
    res = []
    for userid in data.keys():
        # 排除自己计算相似度
        if not userid == userID:
            simliar = Euclidean(userID,userid)
            res.append((userid,simliar))
    res.sort(key=lambda val:val[1])# 按照相似度最大顺序排序
    return res[:4]
RES = top10_simliar('1.0')
print(RES)

error: 
[('17602.0', 0.037535053785096986), ('67346.0', 0.03923924660549805), ('116900.0', 0.03938151824124737), ('130390.0', 0.042373278587501804)]


## 根据相似度来推荐用户

In [45]:
def recommend(user):
    # 相似度最高用户
    top_sim_user = top10_simliar(user)[0][0]
    # 相似度最高用户的观影记录
    items = data[top_sim_user]
    recommendations = []
    # 筛选出该用户未观看的电影病添加到列表中
    for item in items.keys():
        if item not in data[user].keys():
            recommendations.append((item,items[item]))
    recommendations.sort(key=lambda val :val[1],reverse=True) # 按照评分排序
    # 返回评分最高的10部电影
    return recommendations[:10]
Recommend = recommend('1.0')
print(Recommend)

error: 
[('Good Will Hunting (1997)', '5.0'), ('Horton Hears a Who! (2008)', '5.0'), ('Billy Madison (1995)', '5.0'), ('Julie & Julia (2009)', '5.0'), ('Chocolat (2000)', '5.0'), ('Harry Potter and the Order of the Phoenix (2007)', '5.0'), ('"Sisterhood of the Traveling Pants', '5.0'), ('"Secret Life of Bees', '5.0'), ('Happy Gilmore (1996)', '5.0'), ('Big Daddy (1999)', '5.0')]


#### 
但有时我们会碰到因为两个用户之间数据由于数据膨胀，一方数据大，一方数据小，但是两者称明显的线性关系

我们引入Pearson相关系数来衡量两个变量之间的线性相关性。

Pearson：-1~1   -1：完全负相关  1：完全正相关  0：不相关              

相关系数 0.8-1.0 极强相关

0.6-0.8 强相关

0.4-0.6 中等程度相关

0.2-0.4 弱相关

0.0-0.2 极弱相关或无相关

公式：

In [61]:
########################################################################
##计算两用户之间的Pearson相关系数
def pearson_sim(user1,user2):
    # 取出两位用户评论过的电影和评分
    user1_data = data[user1]
    user2_data = data[user2]
    distance = 0
    common = {}
 
    # 找到两位用户都评论过的电影
    for key in user1_data.keys():
        if key in user2_data.keys():
            common[key] = 1
    if len(common) == 0:
        return 0#如果没有共同评论过的电影，则返回0
    n = len(common)#共同电影数目
#     print(n,common)
 
    ##计算评分和
    try:
        sum1 = sum([float(user1_data[movie]) for movie in common])
        sum2 = sum([float(user2_data[movie]) for movie in common])

        ##计算评分平方和
        sum1Sq = sum([pow(float(user1_data[movie]),2) for movie in common])
        sum2Sq = sum([pow(float(user2_data[movie]),2) for movie in common])

        ##计算乘积和
        PSum = sum([float(user1_data[it])*float(user2_data[it]) for it in common])
 
        ##计算相关系数
        num = PSum - (sum1*sum2/n)
        den = sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
    except:
        den = 999
        num = 0
        print('error:') 
    if den == 0:
        return 0
    r = num/den
    return r
 
R = pearson_sim('1.0','3.0')
print(R)

0.22531203182281434


In [63]:
# 计算某个用户与其他用的相似度
def top10_simliar(userID):
    res = []
    for userid in data.keys():
        # 排除自己计算相似度
        if not userid == userID:
            simliar = pearson_sim(userID,userid)
            res.append((userid,simliar))
    res.sort(key=lambda val:val[1])# 按照相似度最大顺序排序
    return res[-4:]

In [64]:
RES = top10_simliar('1.0')
print(RES)

error:
[('79721.0', 1.000000000000017), ('60581.0', 1.0000000000000187), ('83906.0', 1.0000000000000213), ('103682.0', 1.0000000000000255)]


In [65]:
def recommend(user):
    # 相似度最高用户
    top_sim_user = top10_simliar(user)[0][0]
    # 相似度最高用户的观影记录
    items = data[top_sim_user]
    recommendations = []
    # 筛选出该用户未观看的电影病添加到列表中
    for item in items.keys():
        if item not in data[user].keys():
            recommendations.append((item,items[item]))
    recommendations.sort(key=lambda val :val[1],reverse=True) # 按照评分排序
    # 返回评分最高的10部电影
    return recommendations[:10]
Recommend = recommend('1.0')
print(Recommend)

error:
[('"Italian Job', '5.0'), ('"Clockwork Orange', '5.0'), ('RocknRolla (2008)', '5.0'), ('No Country for Old Men (2007)', '5.0'), ('21 Grams (2003)', '5.0'), ('Layer Cake (2004)', '5.0'), ('Seven Pounds (2008)', '5.0'), ('Trainspotting (1996)', '5.0'), ("Carlito's Way (1993)", '5.0'), ('Crash (2004)', '5.0')]
